### 0. Install Libraries

In [2]:
#!pip install -U transformers
#!pip install sentencepiece

# Restart kernel after installation: Runtime -> Restart runtime

     |████████████████████████████████| 2.5MB 7.4MB/s 
     |████████████████████████████████| 901kB 40.9MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 
     |████████████████████████████████| 1.2MB 7.7MB/s 


### 1. Download and load dataset

In [1]:
# Download dataset
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2021-06-26 05:20:31--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  71.4MB/s    in 1.1s    

2021-06-26 05:20:33 (71.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
!ls ac*/

imdbEr.txt  imdb.vocab	README	test  train


In [5]:
!ls ac*/train/

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [7]:
!cat ac*/train/pos/0_9.txt 

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!

In [8]:
from pathlib import Path

def read_imdb_split(split_dir):
    """Helper function to read text from txt files located in 
    `split_dir/pos/*.txt` or `split_dir/neg/*.txt`

    @param split_dir: path to train or test directory that contains both pos and neg subdirectory. 

    @returns texts: List of str where each element is a feature (text)
    @returns labels: List of int where each element is a label (positive:1, negative: 0)
    """

    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ['pos', 'neg']:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(1 if label_dir == 'pos' else 0)

    return texts, labels

train_texts, train_labels = read_imdb_split("aclImdb/train")
test_texts, test_labels = read_imdb_split("aclImdb/test")

In [11]:
print(f"Train size: {len(train_texts)} | Test size: {len(test_texts)}")
print(f"Train size: {len(train_labels)} | Test size: {len(test_labels)}")

Train size: 25000 | Test size: 25000
Train size: 25000 | Test size: 25000


### 2. Split and tokenize dataset

In [12]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [13]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [14]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [24]:
print(train_encodings.keys())

# Each key's value is a list of list. Something like:
# 'input_ids': [[1,2,3], [4,5,6]]
# Refer to the __getitem__ method in the IMDbDataset subclass to see how to access to each element individually.

dict_keys(['input_ids', 'attention_mask'])


### 3. Write Dataset subclasses

In [25]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)        
val_dataset = IMDbDataset(val_encodings, val_labels)        
test_dataset = IMDbDataset(test_encodings, test_labels)        

### 4. Prepare training arguments and start training

In [26]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

Step,Training Loss
10,0.691800
20,0.691300
30,0.700500
40,0.718600
50,0.688500
60,0.688800
70,0.678900
80,0.684200
90,0.664700
100,0.644500


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=7500, training_loss=0.20122720197954524, metrics={'train_runtime': 3150.3136, 'train_samples_per_second': 19.046, 'train_steps_per_second': 2.381, 'total_flos': 1.23411474432e+16, 'train_loss': 0.20122720197954524, 'epoch': 3.0})

**Training completed. Do not forget to share your model on huggingface.co/models =)**

```
TrainOutput(global_step=7500, training_loss=0.20122720197954524, metrics={'train_runtime': 3150.3136, 'train_samples_per_second': 19.046, 'train_steps_per_second': 2.381, 'total_flos': 1.23411474432e+16, 'train_loss': 0.20122720197954524, 'epoch': 3.0})```

### 5. Predicts on Test set

In [27]:
preds = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 25000
  Batch size = 8


In [42]:
predictions = preds[0].argmax(-1)

from sklearn.metrics import classification_report

print(classification_report(preds[1], # labels from test_dataset
                            predictions, 
                            ))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     12500
           1       0.93      0.93      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



### 6. Save model

In [43]:
trainer.save_model("./models")

Saving model checkpoint to ./models
Configuration saved in ./models/config.json
Model weights saved in ./models/pytorch_model.bin


### 7. Inference using pipeline

In [48]:
from transformers import pipeline

review_pipeline = pipeline("text-classification", model="./models", tokenizer=tokenizer, return_all_scores=True)

loading configuration file ./models/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.1",
  "vocab_size": 30522
}

loading configuration file ./models/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max

In [50]:
positive_test_case = "Awesome movie. Love it so much!"

predictions = review_pipeline(positive_test_case)

predictions

[[{'label': 'LABEL_0', 'score': 0.00025538005866110325},
  {'label': 'LABEL_1', 'score': 0.9997446537017822}]]

In [64]:
negative_test_case = "Bad movie and storyline. I hate it so much!"

predictions = review_pipeline(negative_test_case)

predictions

[[{'label': 'LABEL_0', 'score': 0.9973768591880798},
  {'label': 'LABEL_1', 'score': 0.0026231799274683}]]

### Inference using model.from_pretrained()

In [53]:
reload_model = DistilBertForSequenceClassification.from_pretrained("./models")

loading configuration file ./models/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.1",
  "vocab_size": 30522
}

loading weights file ./models/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at ./models.
If your task is similar to the task the model of the checkpoint was trained on, y

In [65]:
device = trainer.args.device

test_cases = [positive_test_case, negative_test_case]
encodings = tokenizer(test_cases, truncation=True, padding=True, return_tensors='pt')

with torch.no_grad():
    outputs = reload_model(**encodings)

predictions = outputs.logits.argmax(-1)

In [66]:
for sent, pred in zip(test_cases, predictions):
    print(f"Sentence: {sent} | Predicted: {'Positive' if pred == 1 else 'Negative'}")

Sentence: Awesome movie. Love it so much! | Predicted: Positive
Sentence: Bad movie and storyline. I hate it so much! | Predicted: Negative


### Reference


1.   Tokenizer: https://huggingface.co/transformers/main_classes/tokenizer.html
2.   Pipeline: https://huggingface.co/transformers/main_classes/pipelines.html#the-pipeline-abstraction
3.   Load model after trainer.train(): https://discuss.huggingface.co/t/how-to-test-my-text-classification-model-after-training-it/6689/2
4.   Fine-tuning with custom datasets: https://huggingface.co/transformers/master/custom_datasets.html
5.   trainer.predict(): https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer.predict
